In [11]:
import pandas as pd
import numpy as np
import jalali
import statistics

# Modify Date And Water Table
# Date Type: 
#   1.Gregorian (Text): 2000-11-1
#   2.Persian (Text): 1399-1-1
# Date & Value:
#   Column of Pandas Dataframe
def convert_to_day_15(info, date, value, date_type="persian"):
    df = info.copy()
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = date.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
    elif date_type == "persian":
        df["DATE_PERSIAN"] = date
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)    
    else:
        pass
    df["VALUE"] = value        
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    df["VALUE_NEW"] = df["VALUE"]
    A = []
    A.append(df["VALUE"][0])
    for i in range(1, len(df)):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    df["VALUE_NEW"] = A
    return df


# Read Data And GeoInfo
xls = pd.ExcelFile('Data/CSV/HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo')

# GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())

# Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())

# Convert Date
Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)

Data['DATE_CHECK'] = np.where(
    Data["DATE_PERSIAN_RAW"].isna(),
    np.where(
        Data["DATE_GREGORIAN_RAW"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)

Data['DATE_PERSIAN_RAW'] = Data.apply(
    lambda x: jalali.Gregorian(x["DATE_GREGORIAN_RAW"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN_RAW"], 
    axis=1
)

Data['DATE_GREGORIAN_RAW'] = Data.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN_RAW"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GREGORIAN_RAW"], 
    axis=1
)

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)


Data.drop(['DATE_CHECK'], axis=1, inplace=True)

Data.sort_values(
    by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"], 
    inplace=True
)

wt_date_converted = convert_to_day_15(
    info=Data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]],
    date=Data["DATE_PERSIAN_RAW"],
    value=Data["WATER_TABLE_RAW"],
    date_type="persian"
)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW"]
)

Data['TMP'] = (Data['STORAGE_COEFFICIENT_LOCATION'] * Data['THISSEN_LOCATION']) / Data['THISSEN_AQUIFER']

tmp = Data.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

Data = Data.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

SELECTED_LEVEL = 'LEVEL_SRTM'

Data = Data.merge(
    right=GeoInfo[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', SELECTED_LEVEL]],
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
).rename(columns={SELECTED_LEVEL: 'LEVEL'})

Data['HEAD_LOCATION'] = Data['LEVEL'] - Data['WATER_TABLE']

Data['TMP'] = (Data['HEAD_LOCATION'] * Data['THISSEN_LOCATION']) / Data['THISSEN_AQUIFER']

tmp = Data.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

Data = Data.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

tmp = Data[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION']]\
    .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .agg({'HEAD_LOCATION': [statistics.mean, statistics.geometric_mean, statistics.harmonic_mean]})\
            .reset_index()

tmp.columns = [col for col in tmp.columns]

tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

Data = Data.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

del tmp


Data

ZeroDivisionError: division by zero